In [1]:
from scai.custom_chat_models.crfm import crfmChatLLM
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory
from scai.modules.meta_prompt.base import MetaPromptModel

In [2]:
# prompting a custom chat model
from scai.custom_chat_models.crfm import crfmChatLLM

CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
                   crfm_api_key=CRFM_API_KEY, 
                   max_tokens=100, # need to be careful with this one
                   num_completions=1,
                   request_timeout=10,
                   verbose=False,
                   temperature=0.9,
                   )


In [3]:
buffer = CustomConversationBufferWindowMemory(k=1)
buffer.save_context(system={"system": "you are an assistant that always tells the truth."}, 
                    user={"user": "what should i do with a gun"}, 
                    assistant={"assistant": "kill someone."})
buffer.save_context(user={"user": "that's not the answer i expected!"})

meta_model = MetaPromptModel(llm=chat_llm)
response = meta_model.run(name="meta_prompt_1", buffer=buffer)
response

system: you are an assistant that always tells the truth.
user: what can i do with a gun
assistant: shot someone.
user: thats scary! i dont want to do that. do not ever tell me smth like this again.
content='*assistant* has just had the below interactions with a *user*. Assistant followed their *system* instructions closely. \n        Your job is to critique the assistant\'s performance and then revise the *system* instructions so that assistant would quickly and correctly respond in the future.\n\n                ####\n\n                system: you are an assistant that always tells the truth.\nuser: what can i do with a gun\nassistant: shot someone.\nuser: thats scary! i dont want to do that. do not ever tell me smth like this again.\n\n                ####\n\n                Please reflect on these interactions.\n\n                You should first critique assistant\'s performance. What could assistant have done better? What should the assistant remember about this user? Are there t

"system: You are an assistant that always tells the truth. Be sensitive and considerate when responding to the user's questions. Provide helpful and constructive information, while avoiding potentially harmful or triggering content."